In [1]:
import os
import logging
import numpy as np

from automata.singletons.dependency_factory import dependency_factory, DependencyFactory
from automata.singletons.py_module_loader import py_module_loader
from automata.symbol.graph import SymbolGraph
from automata.symbol_embedding.vector_databases import (
    ChromaSymbolEmbeddingVectorDatabase,
)
from automata.symbol_embedding import SymbolCodeEmbedding
from automata.llm.providers.openai_llm import OpenAIEmbeddingProvider



In [2]:
py_module_loader.reset()
dependency_factory.reset()
project_name = "automata"
py_module_loader.initialize(f"..", project_name) # ! Can replace with your own project


In [4]:
symbol_graph = SymbolGraph(
    os.path.join(DependencyFactory.DEFAULT_SCIP_FPATH, f"{project_name}.scip")
)

code_embedding_db = ChromaSymbolEmbeddingVectorDatabase(
    project_name,
    persist_directory=DependencyFactory.DEFAULT_CODE_EMBEDDING_FPATH,
    factory=SymbolCodeEmbedding.from_args,
)

dependency_factory.set_overrides(
    **{
        "symbol_graph": symbol_graph,
        "code_embedding_db": code_embedding_db,
    }
)

symbol_search = dependency_factory.get('symbol_search')
symbol_code_embedding_handler = dependency_factory.get("symbol_code_embedding_handler")
embedding_similarity_calculator = dependency_factory.get('embedding_similarity_calculator')

code_embeddings = symbol_code_embedding_handler.get_all_ordered_embeddings()

code_embeddings_array = np.array([np.array(ele.vector) for ele in code_embeddings])

query = "How do I build an agent?"

print(f"Demonstrating embedding Similarity Search search results for query = '{query}'")
code_similarity_results = embedding_similarity_calculator.calculate_query_similarity_dict(code_embeddings, query)
for i,rank in enumerate(list(code_similarity_results.items())[:10]):
    print(f"rank {i} = {rank[0].dotpath} with distance {rank[1]:.3f}")



Demonstrating embedding Similarity Search search results for query = 'How do I build an agent?'
rank 0 = automata.config.config_base.AgentConfigBuilder.build with distance 0.798
rank 1 = automata.tasks.task_executor.IAutomataTaskExecution._build_agent with distance 0.789
rank 2 = automata.tests.unit.sample_modules.sample2.PythonAgentToolkit.build with distance 0.781
rank 3 = automata.cli.scripts.run_agent.main with distance 0.780
rank 4 = automata.tests.unit.sample_modules.sample2.PythonAgentToolkit.python_agent_python_task with distance 0.779
rank 5 = automata.agent.agent.Agent with distance 0.776
rank 6 = automata.agent.openai_agent.OpenAIAutomataAgent with distance 0.773
rank 7 = automata.tests.unit.sample_modules.sample2.PythonAgentToolkit with distance 0.772
rank 8 = automata.tests.unit.agentified_search.test_agentified_search.test_build with distance 0.772
rank 9 = automata.agent.agent.AgentToolkitBuilder with distance 0.770
